# 1、create_sql_agent的使用

举例1：

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit

# 1、获取mysql数据库的连接
# 测试连接本地的mysql数据库
db_user = "root"
db_password = "abc123"
db_host = "localhost" #或 127.0.0.1
db_port = "3306"
db_database = "atguigudb"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

print("操作的是哪种数据库：",db.dialect)
print("获取数据库中的表：",db.get_usable_table_names())

# 执行查询操作
res = db.run("SELECT COUNT(*) FROM employees")
print(res)

举例2：chain的使用

In [ ]:
# 1、获取mysql数据库的连接
# 测试连接本地的mysql数据库
db_user = ""
db_password = ""
db_host = "" # 或 127.0.0.1
db_port = ""
db_database = ""
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

# 2、获取大语言模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")
chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL"))

# 3、创建SQLAgent
chain = create_sql_agent(
    llm=chat_model,
    toolkit=SQLDatabaseToolkit(db=db),
    verbose=True
)

# response = chain.invoke({"question": "数据表employees中一共有多少个员工？",
#                          "table_names_to_use":["employees"]})
# print(response)

response = chain.invoke({"question": "数据表employees中薪资最高的员工信息",
                         "table_names_to_use":["employees"]})
print(response)

# 2、create_stuff_documents_chain的使用

In [ ]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

def format_docs(docs):
    """将文档列表转换为一个字符串"""
    return "\n\n".join(doc.page_content for doc in docs)

# 定义提示词模板
prompt = PromptTemplate.from_template("""
基于以下文档回答问题：

{context}

问题：{input}
""")

# 创建链
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

chain = (
    RunnablePassthrough.assign(context=lambda x: format_docs(x["docs"]))
    | prompt
    | llm
    | StrOutputParser()
)

# 文档输入
docs = [
    Document(
        page_content="苹果，学名Malus pumila Mill.，别称西洋苹果、柰，属于蔷薇科苹果属的植物。苹果是全球最广泛种植和销售的水果之一，具有悠久的栽培历史和广泛的分布范围。苹果的原始种群主要起源于中亚的天山山脉附近，尤其是现代哈萨克斯坦的阿拉木图地区，提供了所有现代苹果品种的基因库。苹果通过早期的贸易路线，如丝绸之路，从中亚向外扩散到全球各地。"
    ),
    Document(
        page_content="香蕉是白色的水果，主要产自热带地区。"

    ),
    Document(
        page_content="蓝莓是蓝色的浆果，含有抗氧化物质。"

    )
]

# 执行摘要
response = chain.invoke({
    "docs": docs,
    "input": "香蕉是什么颜色的？"
})
print(response)